In [59]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [60]:
## I acquire stock list in thailand ###
ex = pd.read_excel('listedCompanies_th_TH.xlsx', sheet_name='listedCompanies_th_TH',skiprows=1)
df = pd.DataFrame(ex['หลักทรัพย์'].astype('str') + '.bk')
df = df.rename(columns={'หลักทรัพย์' : 'ticker'})
df = df.reset_index(drop=True)
df

,ticker
0,24CS.bk
1,2S.bk
2,3BBIF.bk
3,3K-BAT.bk
4,A.bk
...,...
916,YONG.bk
917,YUASA.bk
918,ZAA.bk
919,ZEN.bk


In [61]:
# df = pd.DataFrame(['MO','ASO','HPQ','AMCX','AMLX','CROX','KO'],columns=['ticker'])
# df

### Acquire informations from yfinance function >> .info

In [62]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [63]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'marketCap',
    'currentPrice',
    'enterpriseValue',
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_lastest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


##### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.reset_index(drop=True)


In [64]:
df['sector'].unique()

array(['Basic Materials', 'Industrials', 'Real Estate',
       'Consumer Defensive', 'Utilities', 'Consumer Cyclical',
       'Communication Services', 'Financial Services', 'Healthcare',
       'Technology', 'Energy'], dtype=object)

In [65]:

### Next I perform calculations for MF_ROC and MF_EY ###
df['MF_ROC'] = df['EBIT']/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df['EBIT']/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY
0,2S.bk,2024-08-09,Steel,Basic Materials,1.594989e+09,2.90,1.314299e+09,2.400035e+09,3.039420e+08,1.225020e+08,2024-03-31,0.058443,0.093207
1,3K-BAT.bk,2024-08-09,Electrical Equipment & Parts,Industrials,4.174800e+09,53.25,3.860087e+09,3.736470e+09,8.851644e+08,2.054459e+08,2024-03-31,0.072053,0.053223
2,A5.bk,2024-08-09,Real Estate - Development,Real Estate,3.369080e+09,2.86,4.583295e+09,3.836102e+09,1.114288e+09,6.915935e+08,2024-03-31,0.254093,0.150894
3,AAI.bk,2024-08-09,Packaged Foods,Consumer Defensive,1.253750e+10,5.90,1.192003e+10,5.844300e+09,5.309470e+08,6.072975e+08,2024-03-31,0.114296,0.050948
4,AAV.bk,2024-08-09,Airlines,Industrials,2.904100e+10,2.26,7.137034e+10,7.053564e+10,2.740129e+10,1.790460e+09,2024-03-31,0.041509,0.025087
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,YGG.bk,2024-08-09,Entertainment,Communication Services,7.464763e+08,1.24,7.853679e+08,7.375050e+08,7.321346e+07,6.699039e+07,2024-03-31,0.100845,0.085298
642,YONG.bk,2024-08-09,Building Materials,Basic Materials,9.656000e+08,1.42,1.302733e+09,1.307177e+09,2.661980e+08,1.703912e+08,2024-03-31,0.163684,0.130795
643,YUASA.bk,2024-08-09,Auto Parts,Consumer Cyclical,1.130062e+09,10.50,7.857950e+08,1.646425e+09,3.800130e+08,1.818541e+08,2024-03-31,0.143598,0.231427
644,ZEN.bk,2024-08-09,Restaurants,Consumer Cyclical,1.729473e+09,5.80,2.738745e+09,3.230292e+09,9.035990e+08,2.211030e+08,2024-03-31,0.095029,0.080731


In [66]:
### Ranking regarding MFs####

df['Ranking_MF_ROC'] = df['MF_ROC'].rank()
df['Ranking_MF_EY'] = df['MF_EY'].rank()
df['Ranking_MF'] = df['Ranking_MF_ROC'] + df['Ranking_MF_EY']

df.loc[df['Ranking_MF'] == df['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
244,JAS.bk,2024-08-09,Telecom Services,Communication Services,2.139354e+10,2.58,2.001970e+10,1.799440e+10,4.457690e+09,2.261631e+10,2024-03-31,1.670739,1.129703,646.0,646.0,1292.0


In [67]:
df_sorted = df.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:50]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,JAS.bk,2024-08-09,Telecom Services,Communication Services,2.139354e+10,2.58,2.001970e+10,1.799440e+10,4.457690e+09,2.261631e+10,2024-03-31,1.670739,1.129703,646.0,646.0,1292.0
1,LANNA.bk,2024-08-09,Thermal Coal,Energy,7.402500e+09,14.10,7.844623e+09,1.622346e+10,3.418211e+09,4.825900e+09,2024-03-31,0.376869,0.615186,639.0,641.0,1280.0
2,Q-CON.bk,2024-08-09,Building Products & Equipment,Industrials,3.680000e+09,9.20,2.205099e+09,3.412410e+09,8.513570e+08,1.012475e+09,2024-03-31,0.395336,0.459152,643.0,635.0,1278.0
3,PIN.bk,2024-08-09,Real Estate—Development,Real Estate,5.289600e+09,4.56,6.081488e+09,6.263542e+09,6.600270e+08,1.774394e+09,2024-03-31,0.316657,0.291770,629.0,615.0,1244.0
4,BTC.bk,2024-08-09,Financial Conglomerates,Financial Services,6.020336e+09,0.49,3.441970e+09,4.202732e+09,5.192318e+08,1.046087e+09,2024-03-31,0.283993,0.303921,624.0,618.0,1242.0
5,SRICHA.bk,2024-08-09,Engineering & Construction,Industrials,2.324093e+09,7.50,1.384124e+09,2.911026e+09,6.521662e+08,5.931280e+08,2024-03-31,0.262579,0.428522,609.0,630.0,1239.0
6,JCK.bk,2024-08-09,Real Estate - Development,Real Estate,1.292935e+09,0.34,6.192852e+09,9.097491e+09,3.840008e+09,1.562178e+09,2024-03-31,0.297134,0.252255,627.0,602.0,1229.0
7,SUSCO.bk,2024-08-09,Oil & Gas Refining & Marketing,Energy,3.280000e+09,3.28,5.397212e+09,1.002048e+10,3.305554e+09,1.593605e+09,2024-03-31,0.237323,0.295264,603.0,616.0,1219.0
8,PRI.bk,2024-08-09,Real Estate Services,Real Estate,2.256000e+09,7.05,1.648800e+09,2.572253e+09,6.192820e+08,4.616862e+08,2024-03-31,0.236402,0.280013,602.0,612.0,1214.0
9,CAZ.bk,2024-08-09,Engineering & Construction,Industrials,7.938000e+08,2.70,1.045430e+09,2.631247e+09,1.488775e+09,2.827815e+08,2024-03-31,0.247517,0.270493,605.0,607.0,1212.0
